In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip -q install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.9/454.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.1 MB/s eta 0:0

In [4]:
# 1) Point to your Drive folder
BASE = "/content/drive/MyDrive/ieee-fraud-detection"  # change if your folder name differs

# 2) Import pandas
import pandas as pd

# 3) Read a limited number of rows for fast iteration (lower this if RAM is tight)
train_txn = pd.read_csv(f"{BASE}/train_transaction.csv", nrows=150_000)

# 4) Read identity and merge onto transactions via TransactionID
train_id  = pd.read_csv(f"{BASE}/train_identity.csv")
train_df  = train_txn.merge(train_id, on="TransactionID", how="left")

# 5) Define label and (optionally) reduce dtype
label = "isFraud"
train_df[label] = train_df[label].astype("int8")  # small memory win

# 6) Quick sanity
print(train_df.shape, " | isFraud rate:", round(train_df[label].mean(), 4))
train_df.head(3)


(150000, 434)  | isFraud rate: 0.0265


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Minimal baseline training (kept simple)
from autogluon.tabular import TabularPredictor

label = "isFraud"

# OPTIONAL but sensible: don't use the row identifier as a feature
train_simple = train_df.drop(columns=["TransactionID"])

predictor = TabularPredictor(
    label=label,
    problem_type="binary",
    eval_metric="roc_auc",                 # good for imbalanced data
    path="/content/drive/MyDrive/ag-ieee"  # saves models to Drive
).fit(
    train_simple,
    presets="medium_quality_faster_train", # balanced speed/quality
    time_limit=600,                        # ~10 minutes
    verbosity=2
)

predictor.leaderboard(silent=False);      # see models and validation scores


Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       9.66 GB / 12.67 GB (76.2%)
Disk Space Avail:   58.18 GB / 107.72 GB (54.0%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "/content/drive/MyDrive/ag-ieee"
Train Data Rows:    150000
Train Data Columns: 432
Label Column:       isFraud
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    9984.35 MB
	

[1000]	valid_set's binary_logloss: 0.0495716
[2000]	valid_set's binary_logloss: 0.0496824


	0.9445	 = Validation score   (roc_auc)
	290.93s	 = Training   runtime
	0.72s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 288.09s of the 288.09s of remaining time.
	Fitting with cpus=1, gpus=0, mem=2.5/8.0 GB


[1000]	valid_set's binary_logloss: 0.0461794


	0.9533	 = Validation score   (roc_auc)
	197.95s	 = Training   runtime
	0.49s	 = Validation runtime
Fitting model: RandomForestGini ... Training model for up to 88.39s of the 88.38s of remaining time.
	Fitting with cpus=2, gpus=0, mem=0.1/8.0 GB
	0.9052	 = Validation score   (roc_auc)
	66.45s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: RandomForestEntr ... Training model for up to 19.74s of the 19.74s of remaining time.
	Fitting with cpus=2, gpus=0, mem=0.1/7.9 GB
	Time limit exceeded... Skipping RandomForestEntr.
Fitting model: CatBoost ... Training model for up to 15.07s of the 15.07s of remaining time.
	Fitting with cpus=1, gpus=0, mem=3.1/7.7 GB
	Ran out of time, early stopping on iteration 1.
	0.5538	 = Validation score   (roc_auc)
	9.61s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.00s of the 4.35s of remaining time.
	Ensemble Weights: {'LightGBM': 0.875, 'RandomForestGini': 0.125}
	0.9

                 model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.963877     roc_auc       0.588987  264.467154                0.000859           0.061750            2       True          5
1             LightGBM   0.953257     roc_auc       0.489253  197.954490                0.489253         197.954490            1       True          2
2           LightGBMXT   0.944536     roc_auc       0.724552  290.930444                0.724552         290.930444            1       True          1
3     RandomForestGini   0.905151     roc_auc       0.098875   66.450914                0.098875          66.450914            1       True          3
4             CatBoost   0.553752     roc_auc       0.035305    9.614537                0.035305           9.614537            1       True          4


In [6]:
# I take ~10% as a quick validation slice and let AutoGluon compute ROC AUC.
VAL_FRAC = 0.10
val_idx = train_simple.sample(frac=VAL_FRAC, random_state=42).index
val_df  = train_simple.loc[val_idx].copy()  # includes the label column

metrics = predictor.evaluate(val_df, silent=False)  # uses predictor.eval_metric = 'roc_auc'
metrics  # dictionary with 'roc_auc' and timing


Evaluation: roc_auc on test data: 0.999970705367938
Evaluations on test data:
{
    "roc_auc": 0.999970705367938,
    "accuracy": 0.9972666666666666,
    "balanced_accuracy": 0.9481012658227848,
    "mcc": 0.9453537449215763,
    "f1": 0.945260347129506,
    "precision": 1.0,
    "recall": 0.8962025316455696
}


{'roc_auc': np.float64(0.999970705367938),
 'accuracy': 0.9972666666666666,
 'balanced_accuracy': np.float64(0.9481012658227848),
 'mcc': np.float64(0.9453537449215763),
 'f1': 0.945260347129506,
 'precision': 1.0,
 'recall': 0.8962025316455696}

In [7]:
# I display and save the leaderboard for documentation
OUT = "/content/drive/MyDrive/ag-ieee"
lb = predictor.leaderboard(silent=False)   # shows each model and its val score
lb.to_csv(f"{OUT}/leaderboard.csv", index=False)
print("Saved:", f"{OUT}/leaderboard.csv")


                 model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.963877     roc_auc       0.588987  264.467154                0.000859           0.061750            2       True          5
1             LightGBM   0.953257     roc_auc       0.489253  197.954490                0.489253         197.954490            1       True          2
2           LightGBMXT   0.944536     roc_auc       0.724552  290.930444                0.724552         290.930444            1       True          1
3     RandomForestGini   0.905151     roc_auc       0.098875   66.450914                0.098875          66.450914            1       True          3
4             CatBoost   0.553752     roc_auc       0.035305    9.614537                0.035305           9.614537            1       True          4
Saved: /content/drive/MyDrive/ag-ieee/leaderboard.csv


In [ ]:
# Rebuild variables used by the submission cell: test_ids and y_score
BASE = "/content/drive/MyDrive/ieee-fraud-detection"  # adjust if your folder name differs

# Use existing pandas if available; otherwise import
try:
    pd
except NameError:
    import pandas as pd
import numpy as np

# Load trained predictor if it's not already in memory
try:
    predictor
except NameError:
    from autogluon.tabular import TabularPredictor
    predictor = TabularPredictor.load("/content/drive/MyDrive/ag-ieee")

# Load and merge the unlabeled test data
test_txn = pd.read_csv(f"{BASE}/test_transaction.csv")
test_id  = pd.read_csv(f"{BASE}/test_identity.csv")
test_df  = test_txn.merge(test_id, on="TransactionID", how="left")

# Keep the identifier; drop it from features to match training
test_ids = test_df["TransactionID"].copy()
X_test   = test_df.drop(columns=["TransactionID"])

# Predict positive-class (fraud) probabilities
proba = predictor.predict_proba(X_test)
if hasattr(proba, "iloc"):  # pandas DataFrame/Series case
    y_score = proba[1].to_numpy() if 1 in getattr(proba, "columns", []) else proba.iloc[:, -1].to_numpy()
else:                        # numpy array case
    proba = np.asarray(proba)
    y_score = proba[:, -1] if proba.ndim == 2 else proba

print("Ready for submission cell. Shapes -> X_test:", X_test.shape, "| y_score:", y_score.shape)
